<a href="https://colab.research.google.com/github/ANKITPODDER2000/CNN_Colab/blob/master/VGG16Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Version of tf is : " , tf.__version__)

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [ ]:
dir = "/content/drive/My Drive/CNN/Dog&Cat/data/dataset/"
for i in os.listdir(dir):
  for j in os.listdir(dir+i):
    print("No of images in %s is : %d"%((i+j) , len(os.listdir(dir+i+"/"+j))))

In [ ]:
conv_base = VGG16(weights = "imagenet" ,
                  include_top = False ,
                  input_shape = (150 , 150 , 3))

In [ ]:
conv_base.summary()

In [ ]:
train_dir = "/content/drive/My Drive/CNN/Dog&Cat/data/dataset/training_set/"
test_dir  = "/content/drive/My Drive/CNN/Dog&Cat/data/dataset/test_set/"

In [ ]:
train_data_aug = ImageDataGenerator(rescale = (1.0 / 255.0) ,
                                    rotation_range = 40 ,
                                    height_shift_range = 0.2 ,
                                    width_shift_range = 0.2 ,
                                    shear_range = 0.2 ,
                                    zoom_range = 0.2 ,
                                    horizontal_flip = True ,
                                    fill_mode = "nearest")

test_data_aug  = ImageDataGenerator(rescale = (1.0 / 255.0) )

train_data_aug = train_data_aug.flow_from_directory(train_dir ,
                                                    batch_size = 20 ,
                                                    target_size = (150 , 150) ,
                                                    class_mode = "binary")

test_data_aug = test_data_aug.flow_from_directory(test_dir ,
                                                  batch_size = 20 ,
                                                  target_size = (150 , 150) ,
                                                  class_mode = "binary")


In [ ]:
_ ,ax = plt.subplots(1,5,figsize = (20,5))
for i in range(5):
  ax[i].imshow(train_data_aug[0][0][0])

In [ ]:
model1 = tf.keras.models.Sequential()
model1.add(conv_base)
model1.add(tf.keras.layers.Flatten())
model1.add(tf.keras.layers.Dense(256 , activation = "relu"))
model1.add(tf.keras.layers.Dense(256 , activation = "relu"))
model1.add(tf.keras.layers.Dense(1 , activation = "sigmoid"))
model1.summary()

In [ ]:
conv_base.trainable = False 

In [ ]:
model1.summary()

In [ ]:
model1.compile(optimizer = "rmsprop" , loss = tf.keras.losses.binary_crossentropy , metrics = ['acc'])

In [ ]:
class myCallbacks(tf.keras.callbacks.Callback):
  def on_epoch_end(self , epoch , logs = {}):
    if logs.get('acc')>=0.98:
      self.model.stop_training = True
      print("Get 98% acc on train data!")

In [ ]:
callbacks = myCallbacks
epochs = 30
his1 = model1.fit(train_data_aug ,
                  steps_per_epoch = train_data_aug.n // train_data_aug.batch_size ,
                  epochs = epochs ,
                  validation_data = test_data_aug ,
                  validation_steps = test_data_aug.n // test_data_aug.batch_size,
                  callbacks = [callbacks()])

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
def plot_his(history):
  _ , ax = plt.subplots(1 , 2 ,figsize = (18 , 6))
  ax[0].plot(history.history['acc'] , label = "Training data")
  ax[0].plot(history.history['val_acc'] , label = "Testing data")
  ax[0].set_title("Accurecy")
  ax[0].legend()

  ax[1].plot(history.history['loss'] , label = "Training data")
  ax[1].plot(history.history['val_loss'] , label = "Testing data")
  ax[1].set_title("Loss")
  ax[1].legend()

In [ ]:
plot_his(his1)

In [ ]:
conv_base.trainable = True

In [ ]:
train = False
for layer in conv_base.layers:
  if layer.name == 'block5_conv1':
    train = True
  layer.trainable = train

In [ ]:
model2 = tf.keras.models.Sequential()
model2.add(conv_base)
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(256 , activation = "relu"))
model2.add(tf.keras.layers.Dense(256 , activation = "relu"))
model2.add(tf.keras.layers.Dense(1 , activation = "sigmoid"))

model2.compile(loss='binary_crossentropy',
               optimizer=tf.keras.optimizers.RMSprop(lr=1e-5) ,
               metrics=['acc'])

model2.summary()

In [ ]:
callbacks = myCallbacks
epochs = 50
his2 = model2.fit(train_data_aug ,
                  steps_per_epoch = train_data_aug.n // train_data_aug.batch_size ,
                  epochs = epochs ,
                  validation_data = test_data_aug ,
                  validation_steps = test_data_aug.n // test_data_aug.batch_size,
                  callbacks = [callbacks()])

In [ ]:
plot_his(his2)

In [ ]:
img = tf.keras.utils.get_file("dog1.jpg" , "https://post.medicalnewstoday.com/wp-content/uploads/sites/3/2020/02/322868_1100-800x825.jpg")

In [ ]:
img_pic = cv2.imread(img)
print("Shape of img : " , img_pic.shape)

new_img = cv2.resize(img_pic , (150 , 150))
print(new_img.shape)

plt.imshow(new_img)

In [ ]:
new_img = new_img / 255.0

In [ ]:
model2.predict(new_img[np.newaxis , ])

In [ ]:
rev_pred = {}
for k , v in train_data_aug.class_indices.items():
  rev_pred[v] = k

In [ ]:
def prediction(p,link,exten,print_details = False):
  file_name = "animal"+str(random.randint(1,100000))+"."+exten
  img = tf.keras.utils.get_file(file_name , link)
  img_pic = cv2.imread(img)
  if print_details:
    print("Shape of img : " , img_pic.shape)
  new_img = cv2.resize(img_pic , (150 , 150))
  if print_details:
    print(new_img.shape)
  new_img = new_img / 255.0
  pred = model2.predict_classes(new_img[np.newaxis , ])[0,0]
  
  p.imshow(new_img)
  p.grid(False)
  p.set_title(rev_pred[pred])

In [ ]:
prediction("https://post.medicalnewstoday.com/wp-content/uploads/sites/3/2020/02/322868_1100-800x825.jpg" , "jpg")

In [ ]:
prediction("https://static.toiimg.com/photo/msid-67586673/67586673.jpg?resizemode=4&width=400" , "jpg")

In [ ]:
model2.predict_classes(new_img[np.newaxis , ])[0,0]

In [ ]:
link = [
  ["https://cdn.vox-cdn.com/thumbor/7TCV657-4hrkIYtUeJ3wOh0fI_s=/0x248:4758x2924/1600x900/cdn.vox-cdn.com/uploads/chorus_image/image/42021742/8536633928_62a8d4fb3b_o.0.0.jpg","jpg"],
  ["https://www.thesprucepets.com/thmb/jNFhHl2Uz3SmZrQmrcbCLO4kPu8=/735x0/close-up-of-cat-lying-on-floor-at-home-908763830-1d61bee6961b45ee8a55bdfa5da1ebb3.jpg" , "jpg"],
  ["https://cdn.vox-cdn.com/thumbor/3CLCd-YqNX4yul1wX4zOajKD8ss=/0x0:1280x720/1200x800/filters:focal(538x258:742x462)/cdn.vox-cdn.com/uploads/chorus_image/image/53091739/maxresdefault.0.jpg" , "jpg"],
  ["https://post.medicalnewstoday.com/wp-content/uploads/sites/3/2020/02/322868_1100-800x825.jpg" , "jpg"]
]
_ , ax = plt.subplots(1 , 4 , figsize = (18 , 4))
count  = 0
for i in link:
  prediction(ax[count] , i[0] , i[1])
  count += 1